In [ ]:
!apt install zstd
!wget https://github.com/facebook/zstd/releases/download/v1.5.5/zstd-1.5.5.tar.gz -O zstd-1.5.5.tar.gz
!tar -xf zstd-1.5.5.tar.gz
!cd zstd-1.5.5 && make
!mkdir pkgs patches

In [ ]:
import subprocess
import os
def download_pkg(pkg,ver,sig=False):
  arch="x86_64"
  pkg_file="%s-%s-%s.pkg.tar.zst" % (pkg,ver,arch)
  sig_file="%s.sig"%pkg_file
  url="https://archive.archlinux.org/packages/%s/%s/%s" % (pkg[0],pkg,pkg_file)
  if not os.path.exists("pkg/%s"%pkg_file):
    print(subprocess.run(["wget", url, "-O","pkgs/%s"%(pkg_file)], capture_output=True, text=True).stdout)
  if sig and not os.path.exists("patches/%s"%sig_file):
     print(subprocess.run(["wget", "%s.sig"%url, "-O","patches/%s.sig"%(pkg_file)], capture_output=True, text=True).stdout)
  return "pkgs/%s"%(pkg_file)

with open("update_list.txt", "r") as file:
    for delta in file:
      pkg,old_ver,_,new_ver=delta.split()
      old_zst=download_pkg(pkg,old_ver)
      new_zst=download_pkg(pkg,new_ver,True)
      patch_file="patches/%s.patch"%delta.strip()
      zstd='zstd-1.5.5/zstd'
      print(subprocess.run([zstd,"--patch-from", old_zst,"-19",new_zst,"-o",patch_file],
                           capture_output=True, text=True).stdout)
      print(patch_file)

In [ ]:
!tar --exclude='patches/intellij-idea-community-edition 4:2023.1.3-1 -> 4:2023.1.4-1.patch'\
 --exclude='patches/linux-rt-lts 6.1.33.11.realtime1-1 -> 6.1.38.12.realtime1-2.patch'\
 --exclude='patches/linux 6.4.3.arch1-1 -> 6.4.3.arch1-2.patch'\
 --exclude='patches/nyxt 3.3.0-1 -> 3.4.0-1.patch'\
 -cvzf patches.tar patches/

In [ ]:
!rm -rf patches
!rm -rf pkgs

In [ ]:
!du --human-readable patches.tar

In [ ]:
!cp patches.tar drive/MyDrive/